# Hosts: Hello OpenAssetIO

This notebook illustrates how to initialise an OpenAssetIO session, resulting in a `Manager` instance for communication with a particular asset management system's plugin. We then perform a simple query of the manager, to `resolve` a property of an entity.


## Setup

The core OpenAssetIO library, `openassetio`, provides the "verbs" for interaction with a manager. I.e. it is the API we use to communicate with the manager plugin. Most data exchanged with the manager is opaque from the perspective of the core API.

The industry-specific MediaCreation library, `openassetio-mediacreation`, provides the "nouns" for describing the data exchanged with the manager. I.e. it provides the structure of the data exchanged with the manager.

In [1]:
try:
    import openassetio
    import openassetio_mediacreation
except ImportError:
    print("This notebook requires the packages listed in `resources/requirements.txt` to be installed")
    raise

We'll also pull in a helpers library to help us format our output - this is a library just for these notebooks and not part of the OpenAssetIO ecosystem generally.

In [2]:
from resources import helpers

## Bootstrap OpenAssetIO

The code initializing the OpenAssetIO API is known as a "Host".

This section shows how the API is set up for use with a asset management systems. In this notebook, we make use of the puppetable [BasicAssetLibrary](https://github.com/OpenAssetIO/OpenAssetIO-Manager-BAL), which is configured to read from a simple database included in the resources folder.

In [3]:
from openassetio.hostApi import HostInterface, ManagerFactory
from openassetio.log import ConsoleLogger, SeverityFilter
from openassetio.pluginSystem import PythonPluginSystemManagerImplementationFactory

# OpenAssetIO requires the host of the API to identify itself.
class NotebookHostInterface(HostInterface):
    def identifier(self):
        return "org.jupyter.notebook"

    def displayName(self):
        return "Jupyter Notebook"
        
host_interface = NotebookHostInterface()

# We also need to direct log messages that emerge from the
# API and Manager plugins to somewhere visible. This setup
# filters based on severity, and prints to stdout/err.
logger = SeverityFilter(ConsoleLogger())

# Specify that plugins should be loaded via the python plugin
# system.
impl_factory = PythonPluginSystemManagerImplementationFactory(logger)


# We can now use the ManagerFactory to instantiate a suitable
# manager - in this case, the notebook uses a predefined
# BAL library
manager = ManagerFactory.defaultManagerForInterface(
    "resources/hello_openassetio/openassetio_config.toml", 
    host_interface, 
    impl_factory, 
    logger)

# API calls that are part of the same logical "process"
# should share a Context. Making one here simplifies the
# code in the rest of the notebook.
context = manager.createContext()

We now have working `Manager` instance, that we can use to query asset information, and a `Context`. The context is used to correlate all of the API calls we make in this notebook, so the manager knows they are part of the same user session.

## Getting started

We've been given a URI by a colleague, which we need to turn into a strongly-typed `EntityReference` before we can use it to query the asset management system.

This process ensures that only known URIs are passed to any subsequent API calls. The `createEntityReference` method will throw if the input is not known to the manager. There are other forms with different failure behaviours if exceptions aren't your thing.

In [4]:
logo_ref = manager.createEntityReference("bal:///project_artwork/logos/openassetio")

helpers.display_result(repr(logo_ref))

> **Result:**
> `<openassetio.EntityReference bal:///project_artwork/logos/openassetio>`

Now we have an entity reference for our logo, we can use the API to learn more about it.

## Resolving a property


OpenAssetIO supports various types of queries, including for entity relationships and for publishing. See other notebooks and the API documentation for more details.

For this notebook we're simply going to ask the manager for the entity's location on disk. This is where the MediaCreation package comes in to play, allowing us to specify the shape of the data we want from the manager, and to extract that data once the query has completed.

In particular, we make use of the `LocatableContentTrait` to get a URL to the content. The `ResolveAccess` access mode argument tells the manager what our intended usage of the data is - this is especially important for publishing workflows. For now, we just need to `kRead` some pre-existing data.

In [5]:
from openassetio_mediacreation.traits.content import LocatableContentTrait
from openassetio.access import ResolveAccess

entity_data = manager.resolve(logo_ref, {LocatableContentTrait.kId}, ResolveAccess.kRead, context)

locatable_content_trait = LocatableContentTrait(entity_data)

url = locatable_content_trait.getLocation()

helpers.display_result(url)

> **Result:**
> `file:///some/path/to/logo.jpg`

OpenAssetIO philosophy is to always use URLs when specifying external resources, such as files, even for files on disk. As a convenience, OpenAssetIO includes a utility to convert `file://` URLs to and from file paths, on both Windows and POSIX systems. See API docs for more information.

In [6]:
from openassetio.utils import FileUrlPathConverter, PathType

path = FileUrlPathConverter().pathFromUrl(url, PathType.kPOSIX)  # also .pathToUrl

helpers.display_result(path)

> **Result:**
> `/some/path/to/logo.jpg`